In [2]:
import json
import requests
import pandas as pd
import os

   


In [5]:
api_key = os.getenv("rwa_xyz_api")

def fetch_top_rwa_assets(api_key, limit=1000):
    all_results = []
    page = 1
    per_page = 100
    
    while len(all_results) < limit:
        myQuery = json.dumps({
            "sort": {
                "direction": "desc",
                "field": "market_value_dollar"
            },
            "pagination": {
                "page": page,
                "perPage": per_page
            }
        })
        
        response = requests.get(
            "https://api.rwa.xyz/v4/assets",
            params={"query": myQuery},
            headers={"Authorization": f"Bearer {api_key}"}
        )
        
        data = response.json()
        results = data.get("results", [])
        
        all_results.extend(results)
        print(f"✓ ({len(all_results)} total)")
        
        if not results or len(results) < per_page:
            break
        
        page += 1
    
    return all_results[:limit]

top_assets = fetch_top_rwa_assets(api_key, limit=1000)
df = pd.json_normalize(top_assets)
df = df[df['market_value_dollar.val'] > 0].copy()
key_columns = [
    'name', 'ticker', 'underlying_asset_classes',
    'price_dollar.val_7d', 'daily_transfer_volume_dollar.val_7d',
    'market_value_dollar.val',
    'token_count', 'top_holder_concentration.val_7d',
]

existing = [col for col in key_columns if col in df.columns]
df_clean = df[existing].copy()

print(f"\nFinal dataset: {len(df_clean)} rows × {len(df_clean.columns)} columns")
print(df_clean.head(10))
print(df_clean['market_value_dollar.val'].describe())

✓ (100 total)
✓ (200 total)
✓ (300 total)
✓ (400 total)
✓ (500 total)
✓ (600 total)
✓ (700 total)
✓ (800 total)
✓ (900 total)
✓ (1000 total)

Final dataset: 240 rows × 8 columns
                 name      ticker  \
0         Tether USDt        USDT   
1                USDC        USDC   
2  Figure HELOC Token  FIGR_HELOC   
3                USDS        USDS   
4         Ethena USDe        USDe   
5                USD1        USD1   
6          Paypal USD       PYUSD   
7         Tether Gold        XAUT   
8          Paxos Gold        PAXG   
9          Falcon USD        USDf   

                            underlying_asset_classes  price_dollar.val_7d  \
0  [Money Market - Fund, Money Market - RRP, Cryp...             0.999389   
1                                                 []             0.999716   
2                                               None             1.028307   
3  [Structured Credit, Private Credit, Stablecoin...             0.999830   
4  [Derivatives - Short ETH P